In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llmclassificationfinetuning/sample_submission.csv
/kaggle/input/llmclassificationfinetuning/train.csv
/kaggle/input/llmclassificationfinetuning/test.csv


In [2]:
import pandas as pd
train = pd.read_csv('/kaggle/input/llmclassificationfinetuning/train.csv')
test = pd.read_csv('/kaggle/input/llmclassificationfinetuning/test.csv')
train.head()


,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [3]:
label_cols = ["winner_model_a", "winner_model_b", "winner_tie"]
train["label"] = train[label_cols].values.argmax(axis=1)  # 0=A,1=B,2=tie


In [4]:
#Statistical analysis

class_counts = train["label"].value_counts(normalize=True).sort_index()
print("Class distribution (0=A,1=B,2=tie):")
print(class_counts)

print("\nP(A wins):", class_counts.get(0, 0.0))
print("P(B wins):", class_counts.get(1, 0.0))
print("P(tie):   ", class_counts.get(2, 0.0))

#Insight: No position bias

Class distribution (0=A,1=B,2=tie):
label
0    0.349079
1    0.341911
2    0.309011
Name: proportion, dtype: float64

P(A wins): 0.34907876193955845
P(B wins): 0.341910677314404
P(tie):    0.30901056074603755


In [5]:
# Lengths in characters
train["len_a_char"] = train["response_a"].astype(str).str.len()
train["len_b_char"] = train["response_b"].astype(str).str.len()
train["len_diff_char"] = train["len_a_char"] - train["len_b_char"]  # >0 => A longer

# Bucket length differences
bins = [-10_000, -500, -200, -50, 0, 50, 200, 500, 10_000]
labels = [
    "<-500", "-500 to -200", "-200 to -50",
    "-50 to 0", "0 to 50", "50 to 200",
    "200 to 500", ">500"
]
train["len_diff_bucket"] = pd.cut(train["len_diff_char"], bins=bins, labels=labels)

# For each bucket, how often A wins vs B wins vs tie?
bucket_stats = (
    train.groupby("len_diff_bucket")["label"]
    .value_counts(normalize=True)
    .unstack(fill_value=0.0)
    .rename(columns={0: "P(A wins)", 1: "P(B wins)", 2: "P(tie)"})
)
print(bucket_stats)

#Insight: No verbosity bias

label            P(A wins)  P(B wins)    P(tie)
len_diff_bucket                                
<-500             0.240059   0.513706  0.246236
-500 to -200      0.282903   0.385712  0.331385
-200 to -50       0.309931   0.335862  0.354207
-50 to 0          0.275162   0.270237  0.454601
0 to 50           0.295060   0.291722  0.413218
50 to 200         0.346249   0.294470  0.359281
200 to 500        0.403769   0.278232  0.317999
>500              0.516417   0.232123  0.251460


/tmp/ipykernel_13/1319236212.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train.groupby("len_diff_bucket")["label"]
